## Interaktivni Phi 3 Mini 4K Instruct Chatbot s Whisperom

### Uvod:
Interaktivni Phi 3 Mini 4K Instruct Chatbot je alat koji korisnicima omogućuje interakciju s Microsoft Phi 3 Mini 4K instruct demo koristeći tekstualni ili audio unos. Chatbot se može koristiti za razne zadatke, poput prijevoda, ažuriranja vremenske prognoze i prikupljanja općih informacija.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


[Stvorite svoj Huggingface pristupni token](https://huggingface.co/settings/tokens)

Stvorite novi token  
Unesite novo ime  
Odaberite dozvole za pisanje  
Kopirajte token i spremite ga na sigurno mjesto


Sljedeći Python kod obavlja dva glavna zadatka: uvoz modula `os` i postavljanje varijable okruženja.

1. Uvoz modula `os`:
   - Modul `os` u Pythonu omogućuje interakciju s operativnim sustavom. Omogućuje izvođenje raznih zadataka vezanih uz operativni sustav, poput pristupa varijablama okruženja, rada s datotekama i direktorijima itd.
   - U ovom kodu, modul `os` se uvozi pomoću naredbe `import`. Ova naredba omogućuje korištenje funkcionalnosti modula `os` u trenutnom Python skriptu.

2. Postavljanje varijable okruženja:
   - Varijabla okruženja je vrijednost kojoj mogu pristupiti programi koji se izvode na operativnom sustavu. To je način pohrane postavki konfiguracije ili drugih informacija koje mogu koristiti više programa.
   - U ovom kodu, nova varijabla okruženja se postavlja pomoću rječnika `os.environ`. Ključ rječnika je `'HF_TOKEN'`, a vrijednost se dodjeljuje iz varijable `HUGGINGFACE_TOKEN`.
   - Varijabla `HUGGINGFACE_TOKEN` definirana je neposredno iznad ovog isječka koda, i dodjeljuje joj se string vrijednost `"hf_**************"` pomoću sintakse `#@param`. Ova sintaksa se često koristi u Jupyter bilježnicama kako bi se omogućio unos korisnika i konfiguracija parametara direktno u sučelju bilježnice.
   - Postavljanjem varijable okruženja `'HF_TOKEN'`, ona postaje dostupna drugim dijelovima programa ili drugim programima koji se izvode na istom operativnom sustavu.

Sveukupno, ovaj kod uvozi modul `os` i postavlja varijablu okruženja pod nazivom `'HF_TOKEN'` s vrijednošću koja je definirana u varijabli `HUGGINGFACE_TOKEN`.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Ovaj isječak koda definira funkciju pod nazivom clear_output koja se koristi za brisanje izlaza trenutne ćelije u Jupyter Notebooku ili IPythonu. Razmotrimo kod i razumimo njegovu funkcionalnost:

Funkcija clear_output prima jedan parametar nazvan wait, koji je logička vrijednost (boolean). Po zadanim postavkama, wait je postavljen na False. Ovaj parametar određuje treba li funkcija čekati dok novi izlaz ne bude dostupan za zamjenu postojećeg izlaza prije nego što ga obriše.

Sama funkcija koristi se za brisanje izlaza trenutne ćelije. U Jupyter Notebooku ili IPythonu, kada ćelija generira izlaz, poput ispisanog teksta ili grafičkih prikaza, taj izlaz se prikazuje ispod ćelije. Funkcija clear_output omogućuje brisanje tog izlaza.

Implementacija funkcije nije navedena u isječku koda, što je naznačeno s tri točke (...). Tri točke predstavljaju rezervirano mjesto za stvarni kod koji obavlja brisanje izlaza. Implementacija funkcije može uključivati interakciju s API-jem Jupyter Notebooka ili IPythona kako bi se uklonio postojeći izlaz iz ćelije.

Sveukupno, ova funkcija pruža praktičan način za brisanje izlaza trenutne ćelije u Jupyter Notebooku ili IPythonu, olakšavajući upravljanje i ažuriranje prikazanog izlaza tijekom interaktivnih sesija kodiranja.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Izvedite pretvorbu teksta u govor (TTS) koristeći Edge TTS uslugu. Prođimo kroz relevantne implementacije funkcija jednu po jednu:

1. `calculate_rate_string(input_value)`: Ova funkcija prima ulaznu vrijednost i izračunava brzinu govora za TTS glas. Ulazna vrijednost predstavlja željenu brzinu govora, gdje vrijednost 1 označava normalnu brzinu. Funkcija izračunava brzinu tako da oduzme 1 od ulazne vrijednosti, pomnoži rezultat sa 100 i zatim odredi znak na temelju toga je li ulazna vrijednost veća ili jednaka 1. Funkcija vraća brzinu u formatu "{sign}{rate}".

2. `make_chunks(input_text, language)`: Ova funkcija prima ulazni tekst i jezik kao parametre. Dijeli ulazni tekst na dijelove prema pravilima specifičnim za jezik. U ovoj implementaciji, ako je jezik "English", funkcija dijeli tekst na mjestima gdje se nalazi točka ("."), uklanja sve vodeće ili završne razmake, dodaje točku svakom dijelu i vraća filtrirani popis dijelova.

3. `tts_file_name(text)`: Ova funkcija generira naziv datoteke za TTS audio datoteku na temelju ulaznog teksta. Izvršava nekoliko transformacija na tekstu: uklanja završnu točku (ako postoji), pretvara tekst u mala slova, uklanja vodeće i završne razmake te zamjenjuje razmake s donjim crticama. Zatim skraćuje tekst na maksimalno 25 znakova (ako je dulji) ili koristi cijeli tekst ako je prazan. Na kraju generira nasumični niz koristeći modul [`uuid`] i kombinira ga sa skraćenim tekstom kako bi stvorila naziv datoteke u formatu "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Ova funkcija spaja više audio datoteka u jednu. Prima popis putanja audio datoteka i izlaznu putanju kao parametre. Funkcija inicijalizira prazan objekt `AudioSegment` nazvan [`merged_audio`]. Zatim iterira kroz svaku putanju audio datoteke, učitava audio datoteku koristeći metodu `AudioSegment.from_file()` iz biblioteke `pydub` i dodaje trenutnu audio datoteku u objekt [`merged_audio`]. Na kraju, izvozi spojeni audio na specificiranu izlaznu putanju u MP3 formatu.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Ova funkcija izvodi TTS operaciju koristeći Edge TTS uslugu. Prima popis dijelova teksta, brzinu govora, naziv glasa i putanju za spremanje kao parametre. Ako je broj dijelova veći od 1, funkcija kreira direktorij za pohranu pojedinačnih audio datoteka dijelova. Zatim iterira kroz svaki dio, konstruira Edge TTS naredbu koristeći funkciju `calculate_rate_string()`, naziv glasa i tekst dijela te izvršava naredbu koristeći funkciju `os.system()`. Ako je izvršenje naredbe uspješno, dodaje putanju generirane audio datoteke u popis. Nakon obrade svih dijelova, spaja pojedinačne audio datoteke koristeći funkciju `merge_audio_files()` i sprema spojeni audio na specificiranu putanju za spremanje. Ako postoji samo jedan dio, direktno generira Edge TTS naredbu i sprema audio na putanju za spremanje. Na kraju, vraća putanju generirane audio datoteke.

6. `random_audio_name_generate()`: Ova funkcija generira nasumični naziv audio datoteke koristeći modul [`uuid`]. Generira nasumični UUID, pretvara ga u niz, uzima prvih 8 znakova, dodaje ekstenziju ".mp3" i vraća nasumični naziv audio datoteke.

7. `talk(input_text)`: Ova funkcija je glavni ulaz za izvođenje TTS operacije. Prima ulazni tekst kao parametar. Prvo provjerava duljinu ulaznog teksta kako bi odredila je li to duga rečenica (dulja ili jednaka 600 znakova). Na temelju duljine i vrijednosti varijable `translate_text_flag`, određuje jezik i generira popis dijelova teksta koristeći funkciju `make_chunks()`. Zatim generira putanju za spremanje audio datoteke koristeći funkciju `random_audio_name_generate()`. Na kraju, poziva funkciju `edge_free_tts()` za izvođenje TTS operacije i vraća putanju generirane audio datoteke.

Sveukupno, ove funkcije rade zajedno kako bi podijelile ulazni tekst na dijelove, generirale naziv datoteke za audio datoteku, izvele TTS operaciju koristeći Edge TTS uslugu i spojile pojedinačne audio datoteke u jednu audio datoteku.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

Implementacija dviju funkcija: convert_to_text i run_text_prompt, kao i deklaracija dviju klasa: str i Audio.

Funkcija convert_to_text prima audio_path kao ulaz i transkribira audio u tekst koristeći model nazvan whisper_model. Funkcija prvo provjerava je li gpu zastavica postavljena na True. Ako jest, whisper_model se koristi s određenim parametrima kao što su word_timestamps=True, fp16=True, language='English' i task='translate'. Ako je gpu zastavica False, whisper_model se koristi s fp16=False. Dobivena transkripcija zatim se sprema u datoteku pod nazivom 'scan.txt' i vraća kao tekst.

Funkcija run_text_prompt prima poruku i chat_history kao ulaz. Koristi funkciju phi_demo za generiranje odgovora od chatbota na temelju ulazne poruke. Generirani odgovor zatim se prosljeđuje funkciji talk, koja pretvara odgovor u audio datoteku i vraća putanju do datoteke. Klasa Audio koristi se za prikaz i reprodukciju audio datoteke. Audio se prikazuje pomoću funkcije display iz modula IPython.display, a Audio objekt se kreira s parametrom autoplay=True, tako da audio automatski počinje s reprodukcijom. chat_history se ažurira s ulaznom porukom i generiranim odgovorom, te se vraćaju prazan string i ažurirani chat_history.

Klasa str je ugrađena klasa u Pythonu koja predstavlja niz znakova. Pruža razne metode za manipulaciju i rad s stringovima, kao što su capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill i još mnogo toga. Ove metode omogućuju izvođenje operacija poput pretraživanja, zamjene, formatiranja i manipulacije stringovima.

Klasa Audio je prilagođena klasa koja predstavlja audio objekt. Koristi se za kreiranje audio playera u Jupyter Notebook okruženju. Klasa prihvaća razne parametre kao što su data, filename, url, embed, rate, autoplay i normalize. Parametar data može biti numpy niz, lista uzoraka, string koji predstavlja naziv datoteke ili URL, ili sirovi PCM podaci. Parametar filename koristi se za specificiranje lokalne datoteke iz koje se učitavaju audio podaci, a parametar url koristi se za specificiranje URL-a s kojeg se preuzimaju audio podaci. Parametar embed određuje hoće li audio podaci biti ugrađeni pomoću data URI-ja ili referencirani iz izvornog izvora. Parametar rate specificira brzinu uzorkovanja audio podataka. Parametar autoplay određuje hoće li audio automatski početi s reprodukcijom. Parametar normalize specificira hoće li audio podaci biti normalizirani (prilagođeni) na maksimalni mogući raspon. Klasa Audio također pruža metode poput reload za ponovno učitavanje audio podataka iz datoteke ili URL-a, te atribute poput src_attr, autoplay_attr i element_id_attr za dohvaćanje odgovarajućih atributa za audio element u HTML-u.

Sve u svemu, ove funkcije i klase koriste se za transkripciju audio zapisa u tekst, generiranje audio odgovora od chatbota, te prikaz i reprodukciju audio zapisa u Jupyter Notebook okruženju.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden pomoću AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati autoritativnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane ljudskog prevoditelja. Ne preuzimamo odgovornost za bilo kakve nesporazume ili pogrešne interpretacije koje proizlaze iz korištenja ovog prijevoda.
